# Krew Interval Scheduling Algorithm
This is an online ridesharing algorithm, designed to link riders with appropriate drivers based on many different factors such as proximity, seats available and route.

## Section 1: Imports and Dependencies

In [51]:
import datetime as dt
import sys
from ipywidgets import widgets
from IPython.display import display

## Section 2: Object Creation

We need multiple data structures for this algorithm: 
1. RideManager
    - Responsible for distributing rides optimally amongst all drivers
2. Driver
    - Holds unaccepted ride requests and scheduled rides
    - Able to accept rides
    - Has an upper limit on number of seats available
3. RideRequest
    - Has an origin, destination, startTime and endTime

In [115]:
class RideManager:
    
    # RideManager holds a set of drivers to prevent duplicate drivers and provide fast look-ups.
    def __init__(self):
        self.drivers = set()
        
    # Takes an array of Drivers to be added to the RideManager
    def addDrivers(self, drivers):
        for driver in drivers:
            self.drivers.add(driver)
    
    def assignRideRequest(self, rideRequest):
        driverQueue = rideRequest.getDriverQueue()
        bestDriver = None
        bestDriverSeatsRemaining = sys.maxsize
        bestRideId = None
        
        # Check for available rides that have already been accepted by drivers
        for driver in driverQueue:
            rides = driver.getAcceptedRides()
            for rideId in rides.keys():
                ride = rides.get(rideId)
                startTimes = [ x.getStartTime() for x in ride ]
                endTimes = [ x.getEndTime() for x in ride ]
                riders = sum([ x.getNumRiders() for x in ride ])
                latestStartTime = max(startTimes)
                earliestEndTime = min(endTimes)
                
                if rideRequest.getStartTime() >= latestStartTime and rideRequest.getEndTime() <= earliestEndTime:
                    availableSeats = driver.getSeats() - riders
                    if availableSeats == rideRequest.getNumRiders():
                        bestDriver = driver
                        bestDriverSeatsRemaining = 0
                        bestRideId = rideId
                        break  
                    elif availableSeats > rideRequest.getNumRiders():
                        if availableSeats - rideRequest.getNumRiders() < bestDriverSeatsRemaining:
                            bestDriver = driver
                            bestDriverSeatsRemaining = availableSeats - rideRequest.getNumRiders()
                            bestRideId = rideId
            
            if bestDriverSeatsRemaining == 0:
                break
                        
        
        # No existing rides were found, so assign it to the first driver in the queue.
        if bestDriver == None:
            bestDriver = rideRequest.getBestDriver()
            bestDriver.addRideRequest(rideRequest)
        else:
            print("Found a driver: " + str(bestDriver))
            bestDriver.addToRide(bestRideId, rideRequest)
        
    def __str__(self):
        for driver in self.drivers:
            if driver.rideRequests or driver.acceptedRides:
                print(driver.name + ":")
            if driver.rideRequests:
                print("--- Open Ride Requests: ")
                for ride in driver.rideRequests:
                    print("------ Ride " + str(ride))
            if driver.acceptedRides:    
                print("--- Accepted Rides: ")
                for acceptedRide in driver.acceptedRides:
                    print("------ Ride " + str(acceptedRide))
        return ""

In [116]:
class Driver:
    def __init__(self, name, seats = 4):
        self.name = name
        self.seats = seats
        self.acceptedRides = {}
        self.rideRequests = {}
        
    def addRideRequest(self, ride):
        self.rideRequests[ride.getId()] = ride
    
    def acceptRide(self, rideId):
        self.acceptedRides[rideId] = [self.rideRequests[rideId]]
        self.rideRequests.pop(rideId)
        
    def getAcceptedRides(self):
        return self.acceptedRides
    
    def getSeats(self):
        return self.seats
    
    def addToRide(self, acceptedRideId, rideRequest):
        print(self.acceptedRides.get(acceptedRideId))
        self.acceptedRides.get(acceptedRideId).extend([rideRequest])
                    
    def __str__(self):
        return self.name

In [117]:
class RideRequest:
    def __init__(self, ride_id, origin, destination, startTime, endTime, numRiders = 1):
        self.id = ride_id
        self.origin = origin
        self.destination = destination
        self.startTime = startTime
        self.endTime = endTime
        self.numRiders = numRiders
        self.driverQueue = []
        
    def getId(self):
        return self.id
        
    def createDriverQueue(self, drivers):
        self.driverQueue.extend(drivers)
        
    def getDriverQueue(self):
        return self.driverQueue
    
    def getBestDriver(self):
        return self.driverQueue[0]
    
    def getStartTime(self):
        return self.startTime
    
    def getEndTime(self):
        return self.endTime
    
    def getNumRiders(self):
        return self.numRiders
    
    def __str__(self):
        return self.origin + " -> " + self.destination + " | " + self.startTime + " -> " + self.endTime

## Section 3: RideManager and Driver Setup
Create drivers with names and number of seats. Add the drivers to the RideManager to be used for ride distribution.

In [118]:
rideManager = RideManager()
driver1 = Driver('Driver 1', 2)
driver2 = Driver('Driver 2')
driver3 = Driver('Driver 3', 6)
driver4 = Driver('Driver 4')
driver5 = Driver('Driver 5', 3)
rideManager.addDrivers([driver1, driver2, driver3, driver4, driver5])

## Section 4: Create Ride Requests
Create multiple ride requests with different intervals and assign hard-coded driver queues. This will interface with the driver queue algorithm created.

In [119]:
# Dallas -> Austin... 12 P.M. - 3 P.M. (2 Riders)
ride1 = RideRequest(1, "Dallas", "Austin", dt.time(12), dt.time(15), 2)
ride1.createDriverQueue([driver1, driver3])

# Dallas -> Austin... 11 A.M. - 2:30 P.M.(3 Riders)
ride2 = RideRequest(2, "Dallas", "Austin", dt.time(11), dt.time(14, 30), 3)
ride2.createDriverQueue([driver2, driver3, driver5])

# Dallas -> Austin... 11:30 A.M. - 12:30 P.M. (1 Rider)
ride3 = RideRequest(3, "Dallas", "Austin", dt.time(11, 30), dt.time(12, 30))
ride3.createDriverQueue([driver4, driver1, driver3, driver5])

## Section 5: Run the algorithm
Create ride requests and allow the RideManager to allocate the rides appropriately.

In [120]:
print("Booking Ride 1")
print("------------------------------------------------")
rideManager.assignRideRequest(ride1)
print(rideManager)

print("Driver 1 Accepts Ride 1")
print("------------------------------------------------")
driver1.acceptRide(ride1.getId())
print(rideManager)

print("Booking Ride 2")
print("------------------------------------------------")
rideManager.assignRideRequest(ride2)
print(rideManager)

print("Booking Ride 3")
print("------------------------------------------------")
rideManager.assignRideRequest(ride3)
print(rideManager)


Booking Ride 1
------------------------------------------------
Driver 1:
--- Open Ride Requests: 
------ Ride 1

Driver 1 Accepts Ride 1
------------------------------------------------
Driver 1:
--- Accepted Rides: 
------ Ride 1

Booking Ride 2
------------------------------------------------
Driver 2:
--- Open Ride Requests: 
------ Ride 2
Driver 1:
--- Accepted Rides: 
------ Ride 1

Booking Ride 3
------------------------------------------------
Driver 2:
--- Open Ride Requests: 
------ Ride 2
Driver 1:
--- Accepted Rides: 
------ Ride 1
Driver 4:
--- Open Ride Requests: 
------ Ride 3

